In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.level = logging.INFO

from os import path
import numpy as np
from corputil import FileCorpus
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
from sklearn.linear_model import LogisticRegression

In [ ]:
def link(file):
    return path.join('data', 'Politics', file)

sources = { 'Linke' : [link('Linke.txt'), link('Linke_PR.txt'), link('Linke_Fraktion.txt')],
            'SPD' : [link('SPD_Fraktion.txt')],
            'Grüne' : [link('Grüne.txt'), link('Grüne_Fraktion.txt')],
            'FDP' : [link('FDP.txt'), link('FDP_Fraktion.txt')],
            'CDU' : [link('CDU_Fraktion.txt')],
            'NPD' : [link('NPD_Fraktion_Sachsen.txt'), link('NPD_Fraktion_MV.txt'), link('NPD_Jung.txt')] }

